# Convex Optimization Experiments

This notebook executes the convex optimization experiments from the TRAC_CV codebase. It allows you to run experiments with different optimizers on linear and logistic regression tasks, and visualize the results.

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import pandas as pd
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from datetime import datetime

# Make sure the directory structure is in Python path
import sys
sys.path.append(os.getcwd())

## Import Local Modules

We'll import the necessary modules from the convex_experiments directory.

In [ ]:
# Import local modules
from convex_experiments.models.linear_models import LinearRegressionModel, LogisticRegressionModel
from convex_experiments.data.synthetic_data import generate_linear_data, generate_logistic_data
from convex_experiments.optimizers.trac_pytorch import start_trac
from convex_experiments.trainer import train_convex

## Configuration Settings

Define the settings for our experiments. You can modify these parameters to try different configurations.

In [ ]:
# Experiment settings
config = {
    'num_samples': 1000,         # Number of data samples
    'num_features': 20,          # Number of features
    'noise_std': 0.1,            # Noise standard deviation for linear regression
    'num_iterations': 500,       # Number of training iterations
    'seed': 42,                  # Random seed
    'log_interval': 10,          # Log metrics every N iterations
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'learning_rates': {          # Learning rates for different optimizers
        'SGD': 0.01,
        'Adam': 0.01,
        'Adagrad': 0.1,
        'RMSprop': 0.01,
        'TRAC_SGD': 0.01,        # TRAC might adapt/ignore this
        'TRAC_Adam': 0.01,       # TRAC might adapt/ignore this
    },
    'optimizers': ['SGD', 'Adam', 'TRAC_SGD', 'TRAC_Adam'],  # Optimizers to use
    'tasks': ['linear', 'logistic'],                        # Tasks to run
}

# Create logs directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_dir = f'convex_experiments/logs_{timestamp}'
os.makedirs(log_dir, exist_ok=True)
print(f"Logs will be saved to: {log_dir}")

## Set Random Seeds

Ensure reproducibility by setting random seeds.

In [ ]:
def set_seeds(seed):
    """Set random seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    return

set_seeds(config['seed'])
device = torch.device(config['device'])
print(f"Using device: {device}")

## Functions to Run Experiments

In [ ]:
def setup_optimizer(optimizer_name, model, lr):
    """Set up the specified optimizer."""
    # TRAC requires a log file
    trac_dummy_log = os.path.join(log_dir, f"trac_internal_{optimizer_name}_{config['seed']}.log")
    if os.path.exists(trac_dummy_log):
        os.remove(trac_dummy_log)  # Ensure clean start if re-running
    
    if optimizer_name == 'SGD':
        return optim.SGD(model.parameters(), lr=lr)
    elif optimizer_name == 'Adam':
        return optim.Adam(model.parameters(), lr=lr)
    elif optimizer_name == 'Adagrad':
        return optim.Adagrad(model.parameters(), lr=lr)
    elif optimizer_name == 'RMSprop':
        return optim.RMSprop(model.parameters(), lr=lr)
    elif optimizer_name == 'TRAC_SGD':
        optimizer_class = start_trac(log_file=trac_dummy_log, Base=optim.SGD)
        return optimizer_class(model.parameters(), lr=lr)
    elif optimizer_name == 'TRAC_Adam':
        optimizer_class = start_trac(log_file=trac_dummy_log, Base=optim.Adam)
        return optimizer_class(model.parameters(), lr=lr)
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")

def run_experiment(task, optimizer_name):
    """Run a single experiment for the given task and optimizer."""
    print(f"\nRunning experiment: Task={task}, Optimizer={optimizer_name}")
    
    # Set up data
    w_true = None
    if task == 'linear':
        X, y, w_true = generate_linear_data(
            config['num_samples'], 
            config['num_features'], 
            config['noise_std'], 
            config['seed']
        )
        loss_fn = nn.MSELoss()
        model = LinearRegressionModel(config['num_features']).to(device)
    elif task == 'logistic':
        X, y, w_true = generate_logistic_data(
            config['num_samples'], 
            config['num_features'], 
            config['seed']
        )
        loss_fn = nn.BCEWithLogitsLoss()
        model = LogisticRegressionModel(config['num_features']).to(device)
    else:
        raise ValueError(f"Unknown task: {task}")
    
    # Move data to the appropriate device
    X = X.to(device)
    y = y.to(device)
    w_true = w_true.to(device)
    
    # Set up optimizer
    optimizer = setup_optimizer(optimizer_name, model, config['learning_rates'][optimizer_name])
    
    # Train the model
    log_data = train_convex(
        model=model,
        X=X,
        y=y,
        loss_fn=loss_fn,
        optimizer=optimizer,
        num_iterations=config['num_iterations'],
        w_true=w_true,
        log_interval=config['log_interval']
    )
    
    # Save results
    log_df = pd.DataFrame(log_data)
    results_filename = os.path.join(log_dir, f"results_{task}_{optimizer_name}_seed{config['seed']}.csv")
    log_df.to_csv(results_filename, index=False)
    print(f"Results saved to {results_filename}")
    
    return log_df

def run_all_experiments():
    """Run all experiments specified in the config."""
    results = {}
    
    for task in config['tasks']:
        results[task] = {}
        for optimizer in config['optimizers']:
            log_df = run_experiment(task, optimizer)
            results[task][optimizer] = log_df
    
    return results

## Run All Experiments

Now, let's run all the configured experiments. This may take some time.

In [ ]:
# Run all the experiments
experiment_results = run_all_experiments()

## Visualization Functions

Define functions to visualize the results of our experiments.

In [ ]:
# Define consistent colors for optimizers
OPTIMIZER_COLORS = {
    'SGD': '#1f77b4', 
    'Adam': '#ff7f0e',
    'Adagrad': '#2ca02c',
    'RMSprop': '#d62728',
    'TRAC_SGD': '#9467bd', 
    'TRAC_Adam': '#8c564b',
}

# Metrics to plot (y-axis) and their preferred y-axis labels
METRICS_TO_PLOT = {
    'loss': 'Loss',
    'gradient_norm': 'Gradient Norm',
    'distance_sq_to_opt': 'Squared Distance to Optimum',
    'trac_s_sum': 'TRAC Sum(s)',
}

def plot_results_from_dataframes(results, task, x_axis='iteration'):
    """Generate plots directly from the experiment results dataframes."""
    # Set seaborn style
    sns.set_theme(style="darkgrid")
    
    # Combine all optimizer results for this task
    all_data = []
    for optimizer, df in results[task].items():
        df_copy = df.copy()
        df_copy['optimizer'] = optimizer
        all_data.append(df_copy)
    
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Create plots for each metric
    for metric, y_label in METRICS_TO_PLOT.items():
        if metric not in combined_df.columns:
            print(f"Metric '{metric}' not found in data, skipping plot.")
            continue
            
        plt.figure(figsize=(10, 6))
        
        sns.lineplot(
            data=combined_df,
            x=x_axis,
            y=metric,
            hue='optimizer',
            palette=OPTIMIZER_COLORS,
            legend='full'
        )
        
        plt.title(f'{task.capitalize()} Task: {y_label} vs. {x_axis.capitalize()}')
        plt.xlabel(x_axis.capitalize())
        plt.ylabel(y_label)
        plt.legend(title='Optimizer')
        plt.tight_layout()
        
        # Save plot
        plot_filename = os.path.join(log_dir, f"plot_{task}_{metric}_vs_{x_axis}.png")
        plt.savefig(plot_filename)
        print(f"Plot saved to {plot_filename}")
        plt.show()

def plot_all_results(results, x_axis='iteration'):
    """Generate plots for all tasks in the results."""
    for task in results.keys():
        plot_results_from_dataframes(results, task, x_axis)
        
def load_results_from_logs(log_dir):
    """Load experiment results from CSV files in the log directory."""
    results = {}
    
    # Find all CSV files in the log directory
    csv_files = glob.glob(os.path.join(log_dir, "results_*.csv"))
    
    for file_path in csv_files:
        # Extract task and optimizer from filename
        filename = os.path.basename(file_path)
        parts = filename.replace('.csv', '').split('_')
        
        # Assuming format: results_task_optimizer_seedX.csv
        task = parts[1]
        optimizer = '_'.join(parts[2:-1]) if len(parts) > 3 else parts[2].split('seed')[0]
        
        # Initialize nested dictionaries if needed
        if task not in results:
            results[task] = {}
            
        # Load data
        try:
            df = pd.read_csv(file_path)
            results[task][optimizer] = df
            print(f"Loaded results for {task} - {optimizer}")
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
    
    return results

## Generate Plots for Experiment Results

Now, let's visualize the results of our experiments.

In [ ]:
# Plot all results using iteration as x-axis
plot_all_results(experiment_results, x_axis='iteration')

In [ ]:
# Plot all results using time as x-axis
plot_all_results(experiment_results, x_axis='time')

## Loading Results from CSV Files

You can also load and visualize results from previously saved CSV files.

In [ ]:
# If you want to load results from a specific log directory
# previous_log_dir = 'convex_experiments/logs_previous_run'
# loaded_results = load_results_from_logs(previous_log_dir)
# plot_all_results(loaded_results)

## Experiment Analysis

Let's analyze the performance of the different optimizers on each task.

In [ ]:
# Performance analysis function
def analyze_performance(results):
    """Analyze and compare the performance of different optimizers."""
    summary = {}
    
    for task in results.keys():
        summary[task] = {}
        
        for optimizer, df in results[task].items():
            # Get final values (last row)
            final_metrics = df.iloc[-1].to_dict()
            
            # Extract relevant metrics
            summary[task][optimizer] = {
                'final_loss': final_metrics.get('loss', float('nan')),
                'final_gradient_norm': final_metrics.get('gradient_norm', float('nan')),
                'final_distance_sq': final_metrics.get('distance_sq_to_opt', float('nan')),
                'total_time': final_metrics.get('time', float('nan')),
                'final_iteration': final_metrics.get('iteration', float('nan'))
            }
    
    # Create and display summary dataframes
    for task, optimizers_data in summary.items():
        print(f"\n{task.upper()} TASK SUMMARY:")
        task_df = pd.DataFrame.from_dict(optimizers_data, orient='index')
        print(task_df)
    
    return summary

# Analyze the experiment results
performance_summary = analyze_performance(experiment_results)

## Conclusion

This notebook has demonstrated how to run and visualize the convex optimization experiments. You can modify the configuration settings to try different scenarios, such as using different optimizers, changing learning rates, or trying different dataset sizes.